In [1]:
import pandas as pd
import chess
import chess.pgn
import chess.engine
import io


In [2]:
eng = chess.engine.SimpleEngine.popen_uci("../../stockfish/stockfish")

In [3]:
df = pd.read_csv('../data/df_completo.csv')
df.head(3)

,id,Event,Site,Date,White,Black,Result,BlackElo,ECO,WhiteElo,...,B<=25,B<=50,B<=100,B<=200,B<=500,B>500,Wprojection,Bprojection,DifProjection,Plays
0,0,New York,New York,1918,"Morrison, John Stuart","Marshall, Frank James",0-1,NaN,C89,NaN,...,18,10,6,4,2,0,1152,-49307,-48155,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...
1,1168,New York,New York,1918,"Capablanca, Jose Raul","Marshall, Frank James",1-0,NaN,C89,NaN,...,5,7,3,4,0,1,215844,0,215844,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...
2,1772,Brno,Brno,1921,"Sery, S.","Vecsey, Zoltan",1-0,NaN,C89,NaN,...,5,0,1,0,0,1,201848,0,201848,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...


In [4]:
def get_move_index(n, turn='w'):
    if turn.lower() == 'b':
        return 2 * n - 1
    else:
        return 2 * n - 1 - 1

In [5]:
def get_subvariation(row, move_number):
    # fen = 'r1bq1rk1/2p1bppp/p1n2n2/1p1pp3/4P3/1BP2N2/PP1P1PPP/RNBQR1K1 w - - 0 9'
    # board = chess.Board(fen=fen)
    # white_move = 2 * move_number - 1 - 1
    # black_move = 2 * move_number - 1

    pgn = io.StringIO(row['Plays'])

    game = chess.pgn.read_game(pgn)

    mainline = [m.san() for m in game.mainline()]

    row['SubVariation'] = '{0}. {1}'.format(
        move_number,
        mainline[get_move_index(move_number)]
    ) 

    row['Mainline'] = mainline
    
    return row

df = df.apply(lambda row: get_subvariation(row, 9), axis=1)

In [6]:
df.to_csv('../data/df_subvar.csv', index=False)